In [ ]:
from pathlib import Path
import json

In [ ]:
import cv2
import numpy as np

In [ ]:
input_sample_folder = '../data/PCA_2_4'
input_json_folder = '../data/PCA_2_4_landmarks'
output_folder = '../data/PCA_2_4_avg'

In [ ]:
w = 170
h = 240

n = 60

In [ ]:
Path(output_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
def similarity_transform(in_points, out_points):
    s60 = np.sin(60 * np.pi / 180)
    c60 = np.cos(60 * np.pi / 180)

    in_pts = np.copy(in_points).tolist()
    out_pts = np.copy(out_points).tolist()

    xin = c60 * (in_pts[0][0] - in_pts[1][0]) - s60 * \
        (in_pts[0][1] - in_pts[1][1]) + in_pts[1][0]
    yin = s60 * (in_pts[0][0] - in_pts[1][0]) + c60 * \
        (in_pts[0][1] - in_pts[1][1]) + in_pts[1][1]

    in_pts.append([np.int32(xin), np.int32(yin)])

    x_out = c60 * (out_pts[0][0] - out_pts[1][0]) - s60 * \
        (out_pts[0][1] - out_pts[1][1]) + out_pts[1][0]
    y_out = s60 * (out_pts[0][0] - out_pts[1][0]) + c60 * \
    (out_pts[0][1] - out_pts[1][1]) + out_pts[1][1]

    out_pts.append([np.int32(x_out), np.int32(y_out)])

    tform = cv2.estimateAffinePartial2D(np.array([in_pts]), np.array([out_pts]))
    
    return tform[0]

In [ ]:
def rect_contains(rect, point):
    if point[0] < rect[0]:
        return False
    
    elif point[1] < rect[1]:
        return False
    
    elif point[0] > rect[2]:
        return False
    
    elif point[1] > rect[3]:
        return False
    
    return True

def calculate_triangles(rect, points):
    subdiv = cv2.Subdiv2D(rect)

    for p in points:
        subdiv.insert((p[0], p[1]))

    triangle_list = subdiv.getTriangleList()
    delaunay_tri = []

    for t in triangle_list:
        pt = []

        pt.append((t[0], t[1]))
        pt.append((t[2], t[3]))
        pt.append((t[4], t[5]))

        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])

        if rect_contains(rect, pt1) and rect_contains(rect, pt2) and rect_contains(rect, pt3):
            ind = []
            
            for j in range(0, 3):
                for k in range(0, len(points)):
                    if abs(pt[j][0] - points[k][0]) < 1.0 and abs(pt[j][1] - points[k][1]) < 1.0:
                        ind.append(k)
                        
            if len(ind) == 3:
                delaunay_tri.append((ind[0], ind[1], ind[2]))

    return delaunay_tri

In [ ]:
def constrain_point(p, w, h):
    return (min(max(p[0], 0), w - 1), min(max(p[1], 0), h - 1))

def apply_affine_transform(src, src_tri, dst_tri, size):
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))

    dst = cv2.warpAffine(src, warp_mat, (size[0], size[1]), None,
        flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

    return dst

def warp_triangle(img1, img2, t1, t2):
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))

    t1_rect = []
    t2_rect = []
    t2_rect_int = []

    for i in range(0, 3):
        t1_rect.append(((t1[i][0] - r1[0]), (t1[i][1] - r1[1])))
        t2_rect.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))
        t2_rect_int.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))

    mask = np.zeros((r2[3], r2[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(t2_rect_int), (1.0, 1.0, 1.0), 16, 0)

    img1_rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]

    size = (r2[2], r2[3])

    img2_rect = apply_affine_transform(img1_rect, t1_rect, t2_rect, size)
    img2_rect = img2_rect * mask

    img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]] = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]] * ((1.0, 1.0, 1.0) - mask)
    img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]] = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]] + img2_rect

In [ ]:
for cluster in Path(input_sample_folder).iterdir():
    cluster_id = cluster.name
    cluster_photos = {}
    for cluster_photo in cluster.glob('*.jpg'):
        cluster_photo = cluster_photo.name
        
        if not (Path(input_json_folder) / cluster_photo.replace('.jpg', '.json')).exists():
            continue
        
        cluster_landmarks = json.load(open(Path(input_json_folder) / cluster_photo.replace('.jpg', '.json')))
        
        cluster_photos[cluster_photo] = cluster_landmarks
        
    cluster_images = {}
    
    for cluster_photo, cluster_landmarks in cluster_photos.items():
        cluster_images[cluster_photo] = {
            'landmarks': cluster_landmarks,
            'image': cv2.imread(str(Path(input_sample_folder) / cluster_photo))
        }
        
    images_norm = []
    landmarks_norm = []
    
    eyecorner_dst = [
        (np.int32(0.3 * w), np.int32(h / 3)),
        (np.int32(0.7 * w), np.int32(h / 3))
    ]
    
    boundary_pts = np.array([
        (0, 0), (w / 2, 0), (w - 1, 0), (w - 1, h / 2),
        (w - 1, h - 1), (w / 2, h - 1), (0, h - 1), (0, h / 2)
    ])
    
    landmarks_avg = np.array(
        [(0, 0)] * (n + len(boundary_pts)),
        np.float32()
    )

    for record in cluster_images:
        image = cluster_images[record]['image']
        landmarks = cluster_images[record]['landmarks']
        
        eyecorner_src = [landmarks[36], landmarks[45]]
        tform = similarity_transform(eyecorner_src, eyecorner_dst)
        
        img = cv2.warpAffine(image, tform, (w, h))
        
        landmarks_ = np.reshape(np.array(landmarks), (68, 1, 2))
        landmarks = cv2.transform(landmarks_, tform)
        landmarks = np.float32(np.reshape(landmarks, (68, 2)))
        
        landmarks = np.append(landmarks, boundary_pts, axis=0)
        
        landmarks_avg = landmarks_avg + landmarks / len(cluster_images)

        landmarks_norm.append(landmarks)
        images_norm.append(img)
        
    rect = (0, 0, w, h)
    tri = calculate_triangles(rect, np.array(landmarks_avg))      
    
    output = np.zeros((h, w, 3), np.float32())
    
    for i in range(0, len(images_norm)):
        img = np.zeros((h, w, 3), np.float32())
        for j in range(0, len(tri)):
            t_in = []
            t_out = []

            for k in range(0, 3):
                p_in = landmarks_norm[i][tri[j][k]]
                p_in = constrain_point(p_in, w, h)

                p_out = landmarks_avg[tri[j][k]]
                p_out = constrain_point(p_out, w, h)

                t_in.append(p_in)
                t_out.append(p_out)

            warp_triangle(images_norm[i], img, t_in, t_out)

        # Add image intensities for averaging
        output = output + img

    # Divide by num_images to get average
    output = output / len(cluster_images)

    cv2.imwrite(str(Path(output_folder) / f'average_face_{cluster_id}.jpg'), 255 * output)